# cần xem trang báo này bởi vì nó tổng hợp từ nhiều nguồn 
https://baomoi.com/chu-de.epi

In [17]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException, TimeoutException
import pandas as pd
from time import sleep
from datetime import datetime
import os

In [ ]:


def wait_for_element(driver, by, selector, timeout=180, interval=5):
    """
    Chờ đến khi tìm thấy 1 element có text (tối đa timeout giây).
    Nếu lần đầu thì refresh trang.
    """


    try:
        el = driver.find_element(by, selector)
        if el:
            return el
    except (NoSuchElementException, TimeoutException, StaleElementReferenceException):
        pass

    start = time.time()
    first_try = True

    count=1
    while time.time() - start < timeout:
        print(f'thử lại lần thứ {count}, {time.time() - start:.2f}s')
        try:
            if first_try:
                driver.refresh()
                first_try = False

            el = driver.find_element(by, selector)
            if el:   
                return el
        except (NoSuchElementException, TimeoutException, StaleElementReferenceException):
            pass
        
        count+=1

        time.sleep(interval)

    return None


def wait_for_elements(driver, by, selector, timeout=180, interval=5):
    """
    Chờ đến khi tìm thấy >=1 element (không yêu cầu có text).
    Nếu lần đầu thì refresh trang.
    """

    try:
        els = driver.find_elements(by, selector)
        if els:
            return els
    except (NoSuchElementException, TimeoutException, StaleElementReferenceException):
        pass

    start = time.time()
    first_try = True

    count=1
    while time.time() - start < timeout:
        print(f'thử lại lần thứ {count}, {time.time() - start:.2f}s')
        try:
            if first_try:
                driver.refresh()
                first_try = False

            els = driver.find_elements(by, selector)
            if els:  # ít nhất 1 element
                return els
        except (NoSuchElementException, TimeoutException, StaleElementReferenceException):
            pass

        time.sleep(interval)
        count+=1

    return None


In [23]:

def get_detail_info(url):
    driver.get(url)
    sleep(2)

    result = {
        "title": None,
        "content": "",
        "date": None,
        "hour": None,
        "link": url
    }

    content_parts = []  # tạm lưu các đoạn h2, p

    # lấy các phần tử trong detail-mcontent
    print(1)
    elements = wait_for_elements(driver =driver , by = By.CSS_SELECTOR, selector="div[class='detail-mcontent'] > *",timeout=60)
    if elements is None:
        print(2)
        elements = wait_for_elements(driver = driver, by = By.CSS_SELECTOR, selector="div[class='detail__content'] > *",timeout=60)
    if elements is not None:
        for el in elements:
            if el.tag_name == "h1":
                if result["title"] is None:  # chỉ lấy h1 đầu tiên làm title
                    result["title"] = el.text
                continue

            elif el.tag_name == "h2":
                content_parts.append(el.text)
                continue
            
            else:
                try:
                    elements_son = wait_for_elements(driver = el, by = By.CSS_SELECTOR, selector = "div[data-role='content'] > *",timeout=60)
                except:
                    continue
            if elements_son is not None:
                for el_son in elements_son:

                    if el_son.tag_name == "h2":
                        content_parts.append(el_son.text)
                        continue

                    elif el_son.tag_name == "p":
                        content_parts.append(el_son.text)
                        continue

    # nối content thành đoạn văn
    result["content"] = "\n".join(content_parts)

    # lấy date và hour
    time_info_ele = wait_for_element(driver=driver,by=By.CSS_SELECTOR, selector="div[class='detail-info']",timeout=60)
    if time_info_ele is not None:
        time_info = time_info_ele.text.split(' ')
        result["hour"] = time_info[1]
        result["date"] = time_info[0]
    return result

# https://thanglong.chinhphu.vn/nhieu-sieu-thi-tang-khuyen-mai-hoat-dong-xuyen-tet-1032201271121272.htm

# a = get_detail_info("https://thanglong.chinhphu.vn/nhieu-sieu-thi-tang-khuyen-mai-hoat-dong-xuyen-tet-1032201271121272.htm")
# print(a)

In [15]:
def click_menu_item(driver, content_to_click,category_to_click):
    menu_ele = wait_for_elements(driver, By.CSS_SELECTOR, "div[class='header__menu'] > ul > li > a")
    for i in menu_ele:
        if i.text.lower() == content_to_click.lower():
            i.click()
            break


    category_ele = wait_for_elements(driver, By.CSS_SELECTOR, "div[class='list__category']  ul > li")
    for i in category_ele:
        if i.text.lower() == category_to_click.lower():
            i.click()
            return True
    return False

In [12]:
global driver
driver = webdriver.Chrome()
driver.get("https://baochinhphu.vn/kinh-te/chung-khoan.htm")

In [26]:

content_to_click,category_to_click = ['kinh tế'], ['ngân hàng', 'doanh nghiệp','chứng khoán']
for ctnt in content_to_click:
    for categr in category_to_click:
        clicked = click_menu_item(driver, ctnt, categr)
        if not clicked:
            print(clicked)
            sleep(2)
            continue

        # --- Lấy links ---
        links = []
        for i in ['list__focus', 'list__listing','AjaxLoadBoxVideoOnListPage']:
            link_elements = wait_for_elements(driver, By.CSS_SELECTOR, f"div[class='{i}'] a",10)
            if link_elements is not None:
                for el in link_elements:
                        links.append(el.get_attribute("href"))
        links = list(set(links))
        
        # --- Thu thập dữ liệu ---
        data = []
        for idx, i in enumerate(links[0:30]):
            try:

                content = get_detail_info(i)
            except Exception as e:
                print(f"Lỗi khi lấy nội dung bài {idx}: {i}")
                continue
            data.append(content)

            print(content["title"])
            print(content["date"], content["hour"])
            print(content["content"])
            print(f"Thực hiện lấy nội dung xong bài thứ {idx}",'='* 50,sep='')



        # Lấy ngày giờ hiện tại
        now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
            # --- Lưu CSV bằng pandas ---
        df = pd.DataFrame(data)
        os.makedirs("newspaper_data", exist_ok=True)  # tạo thư mục nếu chưa có
        df.to_csv(f"newspaper_data/{ctnt}_{categr}_{now}.csv", index=False, encoding="utf-8-sig")  # utf-8-sig để mở Excel không lỗi tiếng Việt

thử lại lần thứ 1, 0.00s
thử lại lần thứ 2, 5.32s
1
thử lại lần thứ 1, 0.00s
thử lại lần thứ 2, 5.52s
thử lại lần thứ 3, 10.58s
thử lại lần thứ 4, 15.63s
thử lại lần thứ 5, 20.65s
thử lại lần thứ 6, 25.67s
thử lại lần thứ 7, 30.69s
thử lại lần thứ 8, 35.74s
thử lại lần thứ 9, 40.80s
thử lại lần thứ 10, 45.82s
thử lại lần thứ 11, 50.85s
thử lại lần thứ 12, 55.87s
2
thử lại lần thứ 1, 0.00s
thử lại lần thứ 1, 0.00s
thử lại lần thứ 1, 0.00s
None
06/02/2025 4:39
Tính đến cuối tháng 1/2025, tỷ lệ nợ xấu của các tổ chức tín dụng (TCTD) chiếm 1,83% trong tổng dư nợ. Tình hình thanh khoản của các TCTD trên địa bàn đảm bảo.
Về dư nợ cho vay theo chương trình tín dụng trên địa bàn Thành phố, cho vay theo chương trình kết nối ngân hàng - doanh nghiệp chiếm 12,57% trong tổng dư nợ; cho vay đối với doanh nghiệp nhỏ và vừa chiếm 18,87%; chương trình nông nghiệp, nông thôn chiếm 8,88%; cho vay xuất khẩu chiếm 5,13%; cho vay công nghiệp hỗ trợ chiếm 2,31%; cho vay doanh nghiệp ứng dụng công nghệ cao c

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=139.0.7258.129)
Stacktrace:
	GetHandleVerifier [0x0x7ff6ec1978d5+2802741]
	GetHandleVerifier [0x0x7ff6ebefeb70+79568]
	(No symbol) [0x0x7ff6ebc9c0fa]
	(No symbol) [0x0x7ff6ebc74601]
	(No symbol) [0x0x7ff6ebd2207e]
	(No symbol) [0x0x7ff6ebd42752]
	(No symbol) [0x0x7ff6ebd1aa63]
	(No symbol) [0x0x7ff6ebce3c91]
	(No symbol) [0x0x7ff6ebce4a23]
	GetHandleVerifier [0x0x7ff6ec1c2ced+2979917]
	GetHandleVerifier [0x0x7ff6ec1bd0f3+2956371]
	GetHandleVerifier [0x0x7ff6ec1dacbd+3078173]
	GetHandleVerifier [0x0x7ff6ebf1836e+184014]
	GetHandleVerifier [0x0x7ff6ebf2024f+216495]
	GetHandleVerifier [0x0x7ff6ebf070c4+113700]
	GetHandleVerifier [0x0x7ff6ebf07279+114137]
	GetHandleVerifier [0x0x7ff6ebeedf78+10968]
	BaseThreadInitThunk [0x0x7ff8c38de8d7+23]
	RtlUserThreadStart [0x0x7ff8c401c34c+44]


In [25]:
for i in links:
    print(i)

https://baochinhphu.vn/kinh-te/ngan-hang.htm
https://baochinhphu.vn/gian-khoan-pvd8-bieu-tuong-tinh-than-sang-tao-kien-cuong-va-khat-vong-vuon-xa-cua-pv-drilling-102250818122142257.htm
https://baochinhphu.vn/3-thanh-pho-cua-viet-nam-thang-hang-ngoan-muc-tren-ban-do-khoi-nghiep-toan-cau-102250528202540604.htm
https://baochinhphu.vn/giao-dich-hnx-thang-7-but-pha-nha-dau-tu-nuoc-ngoai-mua-rong-hon-547-ty-dong-102250812181438254.htm
https://baochinhphu.vn/tang-toc-hien-dai-hoa-ban-le-masan-dau-tu-manh-vao-cong-nghe-va-chuoi-cung-ung-tich-hop-102250710173037365.htm
https://baochinhphu.vn/amway-viet-nam-tiep-tuc-nhan-giai-thuong-noi-lam-viec-tot-nhat-chau-a-2025-10225081816464979.htm
https://baochinhphu.vn/trung-tam-du-lieu-quoc-gia-dat-uptime-tier-iv-giai-doan-thiet-ke-dau-tien-tai-viet-nam-102250815162728675.htm
https://baochinhphu.vn/pho-thu-tuong-ho-duc-phoc-hop-ve-du-thao-nghi-dinh-sua-doi-bo-sung-mot-so-dieu-cua-nghi-dinh-so-155-2020-nd-cp-102250813175750452.htm
https://baochinhphu.vn/